due to convnets' ability to operate convolutionallym extracting features from local input patches and allowing for representationi modularity and data efficiency, it also make them highly relevant to sequence processing; time can be treated as a spatial dimension, like the height or width of a 2D image

such 1D convnets can be competitive with RNNs on certain sequence-processing problems, usually at a considerably cheaper computational cost; 1D convnets typically used with dilated kernels, have been used with great success for audio generation and machine translation, it also can offer a fast alternative to RNNs for simple tasks such as text classificaiton and timeseries forecasting

### Understanding 1D Convolution for Sequence Data

![How 1D Convolution Works](./1d_convnet.png)

such 1D convolution layers can recognize local patterns in a sequence, because the same input transformation is performed on every patch, a pattern learned at a certain position in a sentence can later be recognized at a different position, making 1D convnets translation invariant (for temporal translations)

### 1D Pooling for Sequence Data

extracting 1D patches (subsequences) from an input and outputting the maximum value(max pooling) or average value (average pooling)

### Implementing a 1D Convnet

'Conv1D' layer takes as input 3D tensors with shape (samples, times, features) and returns similarly shaped 3D tensors; the convolution window is a 1D window on the temporal axis: axis 1 in the input tensor

In [5]:
# prepare the HS300 data
import os 
import pandas as pd
import numpy as np

data_dir = 'C:/Users/jim_c/Desktop/Task/local_base/benchmark_index'
fname = os.path.join(data_dir, '000300.SH.csv')

df = pd.read_csv(fname, header=0, 
                 usecols=[2,3,4,5,6,7],
                 names=["open", "high", "low", "close", "vol", "amount"])
df = df.reindex(index=df.index[::-1])

output_path='C:/Users/jim_c/Desktop/Task/Literature/Deep_Learning_with_Python/Deep_Learning_in_Practice/deep_learning_for_text_and_sequences/HS300.csv'
df.to_csv(output_path,sep=',',index=True,header=True)

data_dir = 'C:/Users/jim_c/Desktop/Task/Literature/Deep_Learning_with_Python/Deep_Learning_in_Practice/deep_learning_for_text_and_sequences'
fname = os.path.join(data_dir, 'HS300.csv')

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

float_data = np.zeros((len(lines) - 1, len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values

IndexError: index 4439 is out of bounds for axis 0 with size 4439

In [6]:
lookback = 60
step = 5
delay = 1
batch_size = 64

mean = float_data[:4000].mean(axis=0)
float_data -= mean
std = float_data[:4000].std(axis=0)
float_data /= std


# generator yielding timeseries samples an their targets
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size = 64, step=5):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                    i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
                    
        samples = np.zeros((len(rows), 
                            lookback // step,
                            data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets
        
# preparing the training, and test generators

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=4000,
                      shuffle=True,
                      step=step,
                      batch_size=batch_size)

test_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=4001,
                    max_index=None,
                    step=step,
                    batch_size=batch_size)

test_steps = (len(float_data) - 4001 - lookback)

In [35]:
# preparing the training, and test generators

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=4000,
                      shuffle=True,
                      step=step,
                      batch_size=batch_size)

val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=4001,
                    max_index=4400,
                    step=step,
                    batch_size=batch_size)

test_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=4401,
                    max_index=None,
                    step=step,
                    batch_size=batch_size)

train_steps = 4000 - lookback

val_steps = (4400 - 4001 - lookback)

test_steps = (len(float_data) - 4401 - lookback)

In [ ]:
# training and evaluating a simple 1D convnet on the IMDB data
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Conv1D(128, 7, input_shape=(None, float_data.shape[-1])))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPool1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(1))

model.summary()

model.compile(optimizer=RMSprop(lr=1e-4),
              loss='mae',
              metrics=['acc'])

history = model.fit(train_gen,
                    epochs=10,
                    batch_size=128,
                    validation_data=val_gen,
                    validation_steps=val_steps)

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_27 (Conv1D)          (None, None, 128)         5504      
                                                                 
 conv1d_28 (Conv1D)          (None, None, 32)          28704     
                                                                 
 max_pooling1d_9 (MaxPooling  (None, None, 32)         0         
 1D)                                                             
                                                                 
 conv1d_29 (Conv1D)          (None, None, 32)          7200      
                                                                 
 global_max_pooling1d_5 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)               

C:\Users\jim_c\anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
